In [1]:
from itertools import product
import os
import pandas as pd
import shutil

In [2]:
# already reduced to presolve < 5000 x 5000
instance_set = "miplib_2017_5000_v2"
max_runtime = 3600
expected_instances = 10
degrees = [0, 2, 4]
perturbations = ["matrix", "objective", "rhs"]

# Get breakdown of successes and failures by mode

In [3]:
# what I want to know is which error modes account for which incomplete test sets

# running list of strings contained by different error codes
# last two are catchalls
err = {
    "skipping": [],
    f"walltime": [],
    "failed to perturb": [],
    "gurobipy.gurobierror: out of memory": [],
    "fileexistserror": [],
    "vmem": [],
}

warn = {
    "samples of" : [],
    "invalid value encountered in double_scalars": [],
    "could not be solved for": [],
    "presolve failed to reduce": [],
}

# runs that weren't run
no_go = []

# runs that errored out with new error code
other = []

# runs that had no errors
empty = []

# count of instances made
count = {}

for i, file_name in enumerate(os.listdir(os.path.join("instances", instance_set))):
    
    # skip anything not a mip
    if not file_name.endswith(".mps"):
        continue
        
    # get the model name
    stem = file_name[:-4]
    
    # instantiate the count
    count[stem] = {}
    for p, d in product(perturbations, degrees):
        if os.path.isdir(os.path.join("test_sets", instance_set, stem, f"{p}_{d}")):
            count[stem][(p, d)] = len([f for f in os.listdir(os.path.join("test_sets", instance_set, stem, f"{p}_{d}")) if f.endswith(".mps")])
        else:
            count[stem][(p, d)] = 0
        
    for degree in degrees:
        
        stem_degree = f"{stem}_{degree}"
        
        # get the error file path
        err_pth = os.path.join("outfiles_generation", f"{stem_degree}.err")
        
        # check if the series wasn't run
        if not os.path.exists(err_pth):
            no_go.append(stem_degree)
        
        # check if the series ran with no errors or warnings
        elif os.path.getsize(err_pth) == 0:
            empty.append(stem_degree)
            
        else:
            # read in file
            with open(err_pth, "r") as f:
                text = f.read().lower()
                
            # check for error codes
            found_code = False
            for code in err:
                if code in text:
                    err[code].append(stem_degree)
                    found_code = True
                    break
            
            if not found_code:
                for code in warn:
                    if code in text:
                        warn[code].append(stem_degree)
                        found_code = True
                        break
                        
            if not found_code:
                other.append(stem_degree)
                
# count of possible folders
expected_folders = len(degrees) * (i + 1)

In [4]:
no_go

[]

In [5]:
# skipping - could not solve base instance in < 1 hour
len(err[f"skipping"]) / expected_folders

0.2222222222222222

In [6]:
err[f"skipping"]

['neos-2657525-crna_2',
 'neos-2657525-crna_4',
 'seymour_2',
 'seymour_4',
 'xmas10_4',
 'stein9inf_0',
 'stein9inf_2',
 'stein9inf_4',
 'dell_0',
 'dell_2',
 'dell_4',
 'misc05inf_0',
 'misc05inf_2',
 'misc05inf_4',
 'fhnw-binpack4-4_0',
 'fhnw-binpack4-4_2',
 'fhnw-binpack4-4_4',
 'milo-v13-4-3d-4-0_0',
 'milo-v13-4-3d-4-0_2',
 'milo-v13-4-3d-4-0_4',
 'cvs08r139-94_0',
 'cvs08r139-94_2',
 'cvs08r139-94_4',
 'fhnw-binpack4-18_2',
 'fhnw-binpack4-18_4',
 'control30-5-10-4_0',
 'control30-5-10-4_2',
 'control30-5-10-4_4',
 'neos859080_0',
 'neos859080_2',
 'neos859080_4',
 'graph20-20-1rand_0',
 'graph20-20-1rand_2',
 'graph20-20-1rand_4',
 'neos-1420790_0',
 'neos-1420790_2',
 'neos-1420790_4',
 'assign1-10-4_0',
 'assign1-10-4_2',
 'assign1-10-4_4',
 'neos-5261882-treska_0',
 'neos-5261882-treska_2',
 'neos-5261882-treska_4',
 'no-ip-65059_0',
 'no-ip-65059_2',
 'no-ip-65059_4',
 'lotsize_0',
 'lotsize_4',
 'cvs16r128-89_0',
 'cvs16r128-89_2',
 'neos-3530905-gaula_0',
 'neos-3530905-

In [7]:
# perturbation algorithm failed to complete due to not making any perturbations
len(err["failed to perturb"]) / expected_folders

0.0

In [11]:
# ran out of memory
bumpable = set()
print(set(err["gurobipy.gurobierror: out of memory"] + err["vmem"]))
print(len(set(err["gurobipy.gurobierror: out of memory"] + err["vmem"])) / expected_folders)
df = pd.read_csv("more_memory.csv", index_col=0)
for file_degree_name in set(err["gurobipy.gurobierror: out of memory"] + err["vmem"]):
    file_name = file_degree_name.rsplit("_")[0] + ".mps"
    current_memory = df["memory"].get(file_name, 4)  # df.loc[file_name, "memory"]
    if current_memory < 15 and file_name not in bumpable:
        bumpable.add(file_name)
        df.loc[file_name, "memory"] = min(15, current_memory * 2)
print(len(bumpable) / expected_folders)
df["memory"].astype(int)
df.to_csv("more_memory.csv")
for file_name in bumpable:
    if os.path.isdir(os.path.join("test_sets", instance_set, file_name[:-4])):
        # shutil.rmtree(os.path.join("test_sets", instance_set, file_name[:-4]))
        print("removed: ", file_name[:-4])

{'seymour_0', 'neos-1582420_4', 'binkar10_1_0', 'traininstance2_2', 'rocI-4-11_2', '22433_0', 'neos-3592146-hawea_2', 'neos5_0', 'set3-20_2', 'neos-1396125_2', 'cvs16r106-72_0', 'misc07_4', 'neos16_0', 'traininstance2_4', 'prod1_2', 'neos-585192_0', 'nu25-pr12_0', 'gen-ip036_4', 'fillomino7x7-0i_4', 'fastxgemm-n2r6s0t2_4', 'uct-subprob_4', 'neos-1445743_0', 'supportcase26_4', 'neos-555001_4', 'nexp-50-20-4-2_0', 'fastxgemm-n2r7s4t1_2', 'neos16_2', 'ran13x13_2', 'neos-3421095-cinca_0', 'rout_4', 'app1-1_0', 'ran12x21_2', 'mcsched_0', '23588_4', 'neos-3754480-nidda_4', 'nu25-pr12_4', 'csched007_4', 'mappingmesh3x3mpeg2i_4', 'ns2071214_0', 'noswot_2', 'neos5_2', 'neos-3754480-nidda_0', 'cvs16r128-89_4', 'pigeon-10_4', 'neos-1582420_0', 'pg_0', 'set3-20_4', 'pigeon-08_2', 'blp-ir98_2', 'markshare_5_0_2', 'a1c1s1_2', 'csched010_0', 'neos-555884_4', 'neos-3592146-hawea_0', 'mkc1_2', 'uct-subprob_2', 'markshare_5_0_0', 'nh97_potential_2', 'misc07_2', 'gus-sch_4', 'fillomino7x7-0i_2', 'ns20712

In [12]:
# less of an issue - perturbation algorithm failed to complete due to time
len(err[f"walltime"]) / expected_folders

0.017304189435336976

In [13]:
# file exists error - these just need rerun
len(err["fileexistserror"]) / expected_folders

0.0

In [14]:
err["fileexistserror"]

[]

In [15]:
# less of an issue - perturbation algorithm failed to complete due to exhaustion of perturbation attempts
len(warn["samples of"]) / expected_folders

0.39617486338797814

In [16]:
# nonissue
len(warn["invalid value encountered in double_scalars"]) / expected_folders

0.0009107468123861566

In [17]:
# nonissue
len(warn["could not be solved for"]) / expected_folders

0.0

In [18]:
# nonissue
len(warn["presolve failed to reduce"]) / expected_folders

0.0

In [19]:
len(empty) / expected_folders

0.1930783242258652

In [20]:
print(other)
len(other) / expected_folders

[]


0.0

In [18]:
complete = []
removals = []
for stem in count:
    if len([count for (p, d), count in count[stem].items() if count < expected_instances]) == 0:
        complete.append(stem)
    else:
        # remove the directory
        if os.path.isdir(os.path.join("test_sets", instance_set, stem)):
            # shutil.rmtree(os.path.join("test_sets", instance_set, stem))
            removals.append(stem)
len(complete) / expected_folders

0.02459016393442623

In [18]:
len(removals) / expected_folders

0.6420765027322405

In [19]:
for stem in removals:
    for (p, d), amt in count[stem].items():
        if amt < expected_instances:
            print(stem, p, d, amt)

seymour matrix -1 0
seymour matrix 1 0
seymour objective -1 0
seymour objective 1 0
seymour rhs -1 0
seymour rhs 1 0
b1c1s1 rhs 1 0
neos-4650160-yukon matrix 1 0
neos-4650160-yukon rhs -1 3
neos-4650160-yukon rhs 1 0
neos-5075914-elvire matrix -1 0
neos-5075914-elvire matrix 1 0
neos-5075914-elvire objective -1 0
neos-5075914-elvire objective 1 0
neos-5075914-elvire rhs -1 0
neos-5075914-elvire rhs 1 0
neos-1396125 matrix 1 2
neos-1396125 rhs -1 3
neos-1396125 rhs 1 0
rocI-4-11 matrix -1 0
rocI-4-11 matrix 1 0
rocI-4-11 rhs 1 0
hgms-det matrix -1 0
hgms-det matrix 1 0
hgms-det objective -1 0
hgms-det objective 1 0
hgms-det rhs -1 0
hgms-det rhs 1 0
pigeon-16 objective 1 0
pigeon-16 rhs 1 0
xmas10 matrix -1 0
xmas10 matrix 1 0
xmas10 objective -1 0
xmas10 objective 1 0
xmas10 rhs -1 0
xmas10 rhs 1 0
neos-4393408-tinui rhs 1 0
piperout-d27 matrix -1 0
piperout-d27 matrix 1 0
piperout-d27 rhs -1 0
piperout-d27 rhs 1 0
stein9inf matrix -1 0
stein9inf matrix 1 0
stein9inf objective -1 0
ste